In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from operator import itemgetter
import datetime

In [3]:
xls = pd.read_excel('Afsluiters_2018-02-13.xlsx', sheetname=None)
xls_tabs = list(xls.keys())

In [4]:
xls_tabs

['Blad1']

In [5]:
g_afsluiter = xls['Blad1']

In [7]:
g_afsluiter.columns

Index(['ESRI_ID', 'ASSET_NETWERK_NAAM', 'ASSET_NETWERK_ID', 'AANTAL_SLAGEN',
       'BRON_ID', 'DIAMETER', 'DRAAIRICHTING', 'DRUK', 'FABRIKAAT', 'FUNCTIE',
       'JAAR_VAN_EERSTE_AANLEG', 'LANGE_TERMIJN_DATUM', 'NUMMER',
       'OPGETUIGD_JN', 'OPVOER_DATUM', 'PROJECTNUMMER', 'SOORT',
       'SOORT_VERBINDING', 'STAND', 'STATUS_BEHEER', 'TEKENING_ID', 'TYPE',
       'USER_ID', 'VOLLE_DOORLAAT_JN', 'WIJZIG_DATUM', 'IDENTIFICATIENUMMER',
       'AFSTAND_TOT_STATION', 'OPNAME_LIGGING', 'MERIDIAN_URL', 'GELDIG_VAN',
       'GELDIG_TOT', 'ACTUEEL', 'WORKFLOW_ID', 'PROCES_CODE', 'LAAD_DATUM'],
      dtype='object')

In [6]:
g_afsluiter = g_afsluiter[g_afsluiter['Status'] == 'In bedrijf']
g_afsluiter = g_afsluiter.fillna('Onbekend')

KeyError: 'Status'

In [ ]:
# geef de grenzen aan voor de startkwalificaties

# nieuw:     0 t/m 5 jaar
# goed:      6 t/m 20 jaar
# voldoende: 21 t/m 40 jaar
# matig:     ouder dan 40 jaar

kwalificatiejaren = [5, 20, 40]

huidig_jaar = datetime.datetime.now().year

# bereken de jaren die onder een bepaalde specificatie vallen
jaren_in_df = g_afsluiter['Jaar van eerste aanleg'].unique()

jaren_nieuw = [x for x in np.arange(huidig_jaar - kwalificatiejaren[0],
                                    huidig_jaar + 1) if x in jaren_in_df]

jaren_goed = [x for x in np.arange(huidig_jaar - kwalificatiejaren[1],
                                   huidig_jaar - kwalificatiejaren[0]) if x in jaren_in_df]

jaren_voldoende = [x for x in np.arange(huidig_jaar - kwalificatiejaren[2],
                                        huidig_jaar - kwalificatiejaren[1]) if x in jaren_in_df]

jaren_matig = [x for x in np.arange(1800, huidig_jaar - kwalificatiejaren[2]) if x in jaren_in_df]

In [ ]:
def toekennen_startkwalificaties(dataframe, jaren_nieuw, jaren_goed, jaren_voldoende, jaren_matig):

    # toekennen van de startkwalificatie bij elke afsluiter
    voorwaarden = [(dataframe['Jaar van eerste aanleg'].isin(jaren_nieuw)),
                   (dataframe['Jaar van eerste aanleg'].isin(jaren_goed)),
                   (dataframe['Jaar van eerste aanleg'].isin(jaren_voldoende)),
                   (dataframe['Jaar van eerste aanleg'].isin(jaren_matig))]

    startkwalificaties = ['nieuw', 'goed', 'voldoende', 'matig']

    dataframe['startkwalificatie'] = np.select(voorwaarden, startkwalificaties)
    
    return dataframe

In [ ]:
g_afsluiter = toekennen_startkwalificaties(g_afsluiter, jaren_nieuw, jaren_goed, jaren_voldoende, jaren_matig)

In [ ]:
def plot_aantallen_per_jaar(dataframe, jaren_nieuw, jaren_goed, jaren_voldoende, jaren_matig):    
    
    fontsize = 14
    
    jaar_df = pd.DataFrame(dataframe.groupby(['Jaar van eerste aanleg']).size())
    jaar_df.index = jaar_df.index.astype(int)
    jaar_df = jaar_df.rename(columns={0: 'aantal'})
    
    startkwalificatie_aantallen = g_afsluiter.groupby('startkwalificatie').size()
    
    ax = jaar_df.plot(kind='bar', figsize=(20, 10), fontsize=fontsize, color = 'blue', legend=False)

    for i in range(len(jaar_df)):
        if i < len(jaren_matig):
            ax.patches[i].set_facecolor('red')
        if len(jaren_matig) <= i < len(jaren_matig) + len(jaren_voldoende):
            ax.patches[i].set_facecolor('orange')
        if len(jaren_matig) + len(jaren_voldoende) <= i < len(jaren_matig) + len(jaren_voldoende) + len(jaren_goed):
            ax.patches[i].set_facecolor('green')
                      
    plt.title('Totaal aantal afsluiters in gebruik per jaar van aanleg in 2018', fontsize=fontsize+2)
    plt.grid(axis='y')
    plt.xlabel('Jaar' ,fontsize=fontsize)
    plt.ylabel('aantal', fontsize=fontsize)

    plt.axvline(x=len(jaren_matig) - 0.75, color='black', linestyle='--')
    plt.axvline(x=len(jaren_matig)+len(jaren_voldoende) - 0.75, color='black', linestyle='--')
    plt.axvline(x=len(jaren_matig)+len(jaren_voldoende)+len(jaren_goed) - 0.75, color='black', linestyle='--')

    plt.text(len(jaren_matig)/2 - 3, max(jaar_df.aantal)*0.95, ' matig\n '+ str(startkwalificatie_aantallen['matig']),
             fontsize=fontsize+2, color='red')
    plt.text(len(jaren_matig) + 5, max(jaar_df.aantal)*0.95, 'voldoende\n '+ str(startkwalificatie_aantallen['voldoende']),
             fontsize=fontsize+2, color='orange')
    plt.text(len(jaren_matig)+len(jaren_voldoende) + 5, max(jaar_df.aantal)*0.95, 'goed\n '+ str(startkwalificatie_aantallen['goed']),
             fontsize=fontsize+2, color='green')
    plt.text(len(jaren_matig)+len(jaren_voldoende)+len(jaren_goed), max(jaar_df.aantal)*0.95, 'nieuw\n '+ str(startkwalificatie_aantallen['nieuw']),
             fontsize=fontsize+2, color='blue')

    return plt.show()

In [ ]:
plot_aantallen_per_jaar(g_afsluiter, jaren_nieuw, jaren_goed, jaren_voldoende, jaren_matig)

In [ ]:
def toestandsbepaling(dataframe, eigenschap, kwalificatie):
    
    # groepeer per eigenschap
    per_eigenschap = pd.DataFrame(dataframe.groupby([eigenschap, kwalificatie]).size())
    per_eigenschap = per_eigenschap.rename(columns={0: 'aantal'})
    
    # unstack de dataframe en verander ontbrekende waarden in nullen
    per_eigenschap_unstack = per_eigenschap['aantal'].unstack(level=1).fillna(0)
    
    # verander de volgorde van de kolommen
    per_eigenschap_unstack = per_eigenschap_unstack[['nieuw', 'goed', 'voldoende', 'matig']]
    
    # bereken het totaal per eigenschap uit en zet dat in nieuwe kolom 'totaal'
    per_eigenschap_unstack['totaal'] = per_eigenschap_unstack.sum(axis=1)
    
    # sorteer op basis van het totaal aantal per eigenschap
    per_eigenschap_unstack = per_eigenschap_unstack.sort_values(['totaal'])

    # maak een dataframe met percentage bij bepaalde kwalificatie hoort
    per_eigenschap_unstack_relatief = per_eigenschap_unstack.div(0.01 * per_eigenschap_unstack.totaal, axis='index').iloc[:,:-1]

    # plot het resultaat
    fontsize = 14
    per_eigenschap_unstack_relatief.plot(kind = 'barh', figsize=(20, 10), stacked=True,
                                    fontsize=fontsize, color = ['blue','green','orange','red'])
    
    plt.xlabel('Aantal afsluiters (%)', fontsize=fontsize+2)
    plt.ylabel(eigenschap, fontsize=fontsize+2)
    plt.title('Toestandsbepaling afsluiters per eigenschap 2018', fontsize=fontsize+2)

    for i in range(len(per_eigenschap_unstack_relatief)):
        plt.text(100.5, i-0.1, str(int(per_eigenschap_unstack.iloc[i]['totaal'])), fontsize=fontsize)

    plt.legend(loc=8, bbox_to_anchor=(0.5,-0.15), fontsize=fontsize, ncol=4)
    
    return plt.show()

In [ ]:
toestandsbepaling(g_afsluiter, 'Soort', 'startkwalificatie')

In [ ]:
def kwalificatie_aanpassing_door_onbekende_eigenschap(dataframe, eigenschap):
    
    ''' Voeg nieuwe kolom toe genaamd kwalificatie, dat een aanpassing is op de startkwalificatie'''
    
    voorwaarden = [ dataframe[eigenschap].str.contains('Onbekend') == False,
                   (dataframe[eigenschap].str.contains('Onbekend')) & (dataframe['startkwalificatie'] == 'nieuw'),
                   (dataframe[eigenschap].str.contains('Onbekend')) & (dataframe['startkwalificatie'] == 'goed'),
                   (dataframe[eigenschap].str.contains('Onbekend')) & (dataframe['startkwalificatie'] == 'voldoende'),
                   (dataframe[eigenschap].str.contains('Onbekend')) & (dataframe['startkwalificatie'] == 'matig')]

    nieuwe_kwalificaties = [dataframe['startkwalificatie'], 'goed', 'voldoende', 'matig', 'matig']

    dataframe['kwalificatie'] = np.select(voorwaarden, nieuwe_kwalificaties)
    
    return dataframe

In [ ]:
g_afsluiter = kwalificatie_aanpassing_door_onbekende_eigenschap(g_afsluiter, 'G afsluiter type')

In [ ]:
toestandsbepaling(g_afsluiter, 'Soort', 'kwalificatie')

In [ ]:
g_afsluiter.columns

In [ ]:
g_afsluiter.groupby('Jaar van eerste aanleg').size().tail()

In [ ]:

# kenmerken afsluiter is weergegeven met: Functie|Fabrikaat|Soort|Type

# bekijk per jaar van hoeveel afsluiters alles bekend is en hoeveel er ontbrekende data hebben

bekend = g_afsluiter[g_afsluiter['G afsluiter type'].str.contains('Onbekend') == False]
onbekend = g_afsluiter[g_afsluiter['G afsluiter type'].str.contains('Onbekend')]

bekend_per_jaar = pd.DataFrame(bekend.groupby('Jaar van eerste aanleg')['G afsluiter type'].count())
bekend_per_jaar = bekend_per_jaar.rename(columns={'G afsluiter type': 'bekende afsluiters'})

onbekend_per_jaar = pd.DataFrame(onbekend.groupby('Jaar van eerste aanleg')['G afsluiter type'].count())
onbekend_per_jaar = onbekend_per_jaar.rename(columns={'G afsluiter type': 'onbekende afsluiters'})

df_concat = pd.concat([bekend_per_jaar, onbekend_per_jaar], axis=1).fillna(0)

#fontsize = 14
#df_concat.plot(kind='bar', figsize=(20, 10), fontsize=fontsize, color=['green', 'red'])
#plt.title('Kennis afsluiters per jaar van aanleg', fontsize=fontsize)
#plt.grid(axis='y')
#plt.xlabel('Jaartal', fontsize=fontsize)
#plt.ylabel('Aantal', fontsize=fontsize)
#plt.legend(loc=8, bbox_to_anchor=(0.8,0.93), fontsize=fontsize, ncol=2)
#plt.show()

In [ ]:
# kijken welke eigenschappen het vaakst ontbreken

def tel_bekendheid(df, eigenschap):
    onbekenden = df[eigenschap].str.contains('Onbekend').sum()
    bekenden = df[eigenschap].count() - onbekenden
    return bekenden, onbekenden

df_onbekenden = pd.DataFrame(index=['bekenden', 'onbekenden'], columns=['Type', 'Soort', 'Fabrikaat', 'Functie'])

for eigenschap in df_onbekenden.columns.values:
    df_onbekenden[eigenschap] = tel_bekendheid(g_afsluiter, eigenschap)

In [ ]:
df_onbekenden

In [ ]:
# verlaag de kwalificatie als er onbekende data voor die afsluiter is

voorwaarden_2 = [ g_afsluiter['G afsluiter type'].str.contains('Onbekend') == False,
                 (g_afsluiter['G afsluiter type'].str.contains('Onbekend')) & (g_afsluiter['startkwalificatie'] == 'nieuw'),
                 (g_afsluiter['G afsluiter type'].str.contains('Onbekend')) & (g_afsluiter['startkwalificatie'] == 'goed'),
                 (g_afsluiter['G afsluiter type'].str.contains('Onbekend')) & (g_afsluiter['startkwalificatie'] == 'voldoende'),
                 (g_afsluiter['G afsluiter type'].str.contains('Onbekend')) & (g_afsluiter['startkwalificatie'] == 'matig')]

nieuwe_kwalificaties = [g_afsluiter['startkwalificatie'], 'goed', 'voldoende', 'matig', 'matig']

g_afsluiter['kwalificatie'] = np.select(voorwaarden_2, nieuwe_kwalificaties)

In [ ]:
# overzicht van de kwalificaties

kwalificaties = pd.DataFrame(index=['nieuw', 'goed', 'matig', 'voldoende'], columns=['startkwalificatie', 'kwalificatie'])
kwalificaties['startkwalificatie'] = g_afsluiter.groupby(['startkwalificatie']).size()
kwalificaties['kwalificatie'] = g_afsluiter.groupby(['kwalificatie']).size()
kwalificaties

In [ ]:
toestandsbepaling(g_afsluiter, 'Soort', 'kwalificatie')

In [ ]:
# data verrijking probeersels

# kijken wanneer een onbekend fabrikaat er slechts 1 kan zijn afgeleid uit de andere gegevens
type_en_soort_bekend = g_afsluiter[(g_afsluiter['Type'] != 'Onbekend') & (g_afsluiter['Soort'] != 'Onbekend')]
df = type_en_soort_bekend.groupby(['Type', 'Soort', 'Diameter'])['Fabrikaat'].agg(['unique', 'nunique', 'size'])

df3 = type_en_soort_bekend.groupby(['Type', 'Soort', 'Diameter', 'Fabrikaat']).size()
df3 = pd.DataFrame(df3)
df3 = df3.rename(columns={0: 'Aantal'})

g_afsluiter_8bar = g_afsluiter[g_afsluiter['Druk'] == '8 bar']

afsluiters_groepen = pd.DataFrame(g_afsluiter_8bar.groupby(['Diameter', 'Fabrikaat']).size())
afsluiters_groepen = afsluiters_groepen.rename(columns={0: 'Aantal'})

df3